# Computation Graph and Expressions

In [1]:
import dynet as dy

dy.renew_cg()

v1 = dy.inputVector([1,2,3,4])

v2 = dy.inputVector([5,6,7,8])

v3 = v1 + v2

v4 = v3 * 2

v5 = v1 + 1

v6 = dy.concatenate([v1,v2,v3,v5])

print(v6)

print(v6.npvalue())

expression 5/1
[ 1.  2.  3.  4.  5.  6.  7.  8.  6.  8. 10. 12.  2.  3.  4.  5.]


In [5]:
model = dy.Model()

pW = model.add_parameters((20,4))
pb = model.add_parameters(20)

dy.renew_cg()

x = dy.inputVector([1,2,3,4])
W = dy.parameter(pW)
b = dy.parameter(pb)

y = W * x + b

print(y)

print(y.npvalue())

expression 4/5
[-2.05165911 -0.17920166 -0.63342535 -2.28571796  2.33943367  1.44374061
  2.40361357  0.15762496  0.33884883 -0.65287077  4.08012819  1.06572986
 -0.97193158 -1.87402773 -0.03989619  0.8358379  -2.28551912  0.34185609
 -1.28506875  0.76829171]


In [10]:
model = dy.Model()

trainer = dy.SimpleSGDTrainer(model)

p_v = model.add_parameters(10)

EPOCHS = 10

for epoch in range (EPOCHS):
    dy.renew_cg()
    
    v = dy.parameter(p_v)
    v2 = dy.dot_product(v,v)
    v2.forward()
    
    v2.backward()
    
    trainer.update()
    
print(v2.npvalue())

[0.01639034]


In [18]:
# XOR problem

import dynet as dy
import random

data = [([0,1],0),
        ([1,0],0),
        ([0,0],1),
        ([1,1],1)]

model = dy.Model()
pU = model.add_parameters((4,2))
pb = model.add_parameters(4)
pv = model.add_parameters(4)

trainer = dy.SimpleSGDTrainer(model)




EPOCHS = 1000

for epoch in range (EPOCHS):
    epoch_loss = 0.0
    for x,y in data:
        dy.renew_cg()
        U = dy.parameter(pU)
        b = dy.parameter(pb)
        v = dy.parameter(pv)
        x = dy.inputVector(x)
        
        # prediction
        
        yhat = dy.logistic(dy.dot_product(v,dy.tanh(U * x + b)))
        
        # calculate loss
        if y == 0:
            loss = -dy.log(1 - yhat)
        else:
            loss = -dy.log(yhat)
            
        epoch_loss += loss.scalar_value()
        
        loss.backward()
        trainer.update()
        
    print("Epoch %d. loss = %f" % (epoch, epoch_loss))
    
inputData = [0,0]
i = dy.inputVector(inputData)
yhat = dy.logistic(dy.dot_product(v,dy.tanh(U * x + b)))

print(yhat.value())
            
        
        
        
    

Epoch 0. loss = 3.466038
Epoch 1. loss = 3.314329
Epoch 2. loss = 3.206483
Epoch 3. loss = 3.128808
Epoch 4. loss = 3.071772
Epoch 5. loss = 3.028930
Epoch 6. loss = 2.995968
Epoch 7. loss = 2.969995
Epoch 8. loss = 2.949054
Epoch 9. loss = 2.931807
Epoch 10. loss = 2.917322
Epoch 11. loss = 2.904940
Epoch 12. loss = 2.894185
Epoch 13. loss = 2.884714
Epoch 14. loss = 2.876267
Epoch 15. loss = 2.868650
Epoch 16. loss = 2.861714
Epoch 17. loss = 2.855342
Epoch 18. loss = 2.849442
Epoch 19. loss = 2.843940
Epoch 20. loss = 2.838776
Epoch 21. loss = 2.833898
Epoch 22. loss = 2.829266
Epoch 23. loss = 2.824844
Epoch 24. loss = 2.820600
Epoch 25. loss = 2.816508
Epoch 26. loss = 2.812546
Epoch 27. loss = 2.808690
Epoch 28. loss = 2.804924
Epoch 29. loss = 2.801228
Epoch 30. loss = 2.797588
Epoch 31. loss = 2.793989
Epoch 32. loss = 2.790416
Epoch 33. loss = 2.786857
Epoch 34. loss = 2.783299
Epoch 35. loss = 2.779730
Epoch 36. loss = 2.776137
Epoch 37. loss = 2.772509
Epoch 38. loss = 2.768

In [34]:
# Deep Averaging Network (Look-up table)

import dynet as dy
import random
import pandas as pd
import numpy as np
import csv


l2i = {}
i2l = {}

V = 1000
HID = 20
EDIM = 50
NOUT = 5


def predict_labels(doc):
    x = encode_doc(doc)
    h= layer1(x)
    y = layer2(h)
    
    return dy.softmax(y)


def layer1(x):
    W = dy.parameter(pW1)
    b = dy.parameter(pb1)
    
    return dy.tanh(W * x + b)

def layer2(x):
    W = dy.parameter(pW2)
    b = dy.parameter(pb2)
    
    return dy.tanh(W * x + b)

def encode_doc(doc):
    doc = [w2i[w] for w in doc]
    embs = [E[idx] for idx in doc]
    
    return dy.esum(embs)

def do_loss(probs, label):
    label = l2i[label]
    
    return -dy.log(dy.pick(probs,label))

def classify(doc):
    dy.renew_cg()
    probs = predict_labels(doc)
    
    vals = probs.npvalue()
    
    return i2l[np.argmax(vals)]

def read_dataset(fname):
    print('Reading dataset...')
    df = pd.read_csv(fname)

    tags_index = {
        "sci-fi": 1,
        "action": 2,
        "comedy": 3,
        "fantasy": 4,
        "animation": 5,
        "romance": 6,
    }
    
    index_tags = {
        1 : "sci-fi",
        2 : "action",
        3 : "comedy",
        4 : "fantasy",
        5 : "animation",
        6 : "romance",
    }
   
    data = []
    with open(fname) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            else:
                data.append([row[2],tags_index[row[3].strip("\n")]])
                line_count += 1

    return data, tags_index,index_tags

def read_glove():
    print('Loading word vectors...')
    word2vec = {}
    embedding = []
    embeds = []
    word2idx = {}
    with open('glove.6B.50d.txt', encoding='utf-8') as f:
      for line in f:
        values = line.split()
        word = values[0]
        word2idx[word] = len(embeds)
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
        embeds.append(vec)
    return np.array(embeds),word2idx
    
    

    
    
data, l2i, i2l = read_dataset("tagged_plots_movielens.csv")
embedding , w2i = read_glove()

pW1 = model.add_parameters((HID, EDIM))
pb1 = model.add_parameters(HID)
pW2 = model.add_parameters((NOUT, HID))
pb2 = model.add_parameters(NOUT)
E = model.add_lookup_parameters((len(w2idx), EDIM),init =  embedding )

for (doc, label) in data:
    dy.renew_cg()
    probs = predict_labels(doc)
    
    loss = do_loss(probs,label)
    loss.forward()
    loss.backward()
    trainer.update()
    

Reading dataset...
Loading word vectors...


KeyError: 'A'